<a href="https://colab.research.google.com/github/DaBa2000/Pill-Detection-AI/blob/main/Training/Yolo_for_Pills_320.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a align="left" href="https://ultralytics.com/yolov5" target="_blank">
<img src="https://user-images.githubusercontent.com/26833433/125273437-35b3fc00-e30d-11eb-9079-46f313325424.png"></a>

This is the **official YOLOv5 🚀 notebook** authored by **Ultralytics**, and is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/). 
For more information please visit https://github.com/ultralytics/yolov5 and https://ultralytics.com. Thank you!

# Setup

Setting up the YOLOv5 Repository, installing required packages


In [1]:
# Clone YOLOv5 Repo and additional update files
!git clone https://github.com/ultralytics/yolov5  # clone repo
!git clone https://github.com/DaBa2000/Pill-Detection-AI.git

%cd yolov5

# checkout old YOLOv5 Version as newest Version is incompatible wih Barracuda
!git checkout 2c073cd
%pip install -qr requirements.txt 

# overwrite google_utils.py file, as it would load neewest yolo version (v6.0) which is incompatible
%cp -f ../Pill-Detection-AI/Training/google_utils.py utils/google_utils.py


import os
import torch
from IPython.display import Image, clear_output

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 9755, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 9755 (delta 5), reused 1 (delta 1), pack-reused 9742
Receiving objects: 100% (9755/9755), 10.09 MiB | 24.24 MiB/s, done.
Resolving deltas: 100% (6772/6772), done.
Cloning into 'Pill-Detection-AI'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 85 (delta 24), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (85/85), done.
/content/yolov5
Note: checking out '2c073cd'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout co

# Dataset


In [ ]:
# Clone training and testset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Check if dataset is complete for every case
path, dirs, files = next(os.walk("../drive/MyDrive/Data/train1"))
train1_count = len(files)

path, dirs, files = next(os.walk("../drive/MyDrive/Data/train2"))
train2_count = len(files)

path, dirs, files = next(os.walk("../drive/MyDrive/Data/train3"))
train3_count = len(files)

path, dirs, files = next(os.walk("../drive/MyDrive/Data/val"))
val_count = len(files)

if val_count == 174:
  print("Validation set is complete")
else:
  print("Validation set has " + str(val_count) + " istead of 108 files.")

if train1_count == 240:
  print("Training set 1 is complete")
else:
  print("Training set 1 has " + str(train1_count) + " istead of 240 files.")

if train2_count == 20880:
  print("Training set 2 is complete")
else:
  print("Training set 2 has " + str(train2_count) + " istead of 20880 files.")

if train3_count == 41344:
  print("Training set 3 is complete")
else:
  print("Training set 3 has " + str(train3_count) + " istead of 41344 files.")

Validation set is complete
Training set 1 is complete
Training set 2 is complete
Training set 3 is complete


# Setup model 1: Manual Image Training

In [ ]:
# Copy yaml file with arguments for first run
cp ../drive/MyDrive/run1.yaml ./data

In [ ]:
# Train YOLOv5s on custom data
!python train.py --img 320 --batch 8 --epochs 60 --data run1.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=run1.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=60, batch_size=8, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache_images=True, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1
github: ⚠️ WARNING: code is out of date by 238 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 🚀 v5.0-304-g2c073cd torch 1.9.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=

# Setup model 2 Image generation


In [ ]:
# check if files for this run are complete
index = True
for i in range(0, 10):
  for j in range(0, 58):
    for k in range(0, 18):
      if not os.path.isfile("../drive/MyDrive/Data/train2/TestImage" + str(k) + "_" + str(j) + "_" + str(i) + ".jpg"):
        print("../drive/MyDrive/Data/train2/TestImage" + str(k) + "_" + str(j) + "_" + str(i) + ".jpg missing")
        index = False
      if not os.path.isfile("../drive/MyDrive/Data/train2/TestImage" + str(k) + "_" + str(j) + "_" + str(i) + ".txt"):
        print("../drive/MyDrive/Data/train2/TestImage" + str(k) + "_" + str(j) + "_" + str(i) + ".txt missing")
        index = False

if index:
  print("All " + str(train2_count) + " images found")

All 20880 images found


In [ ]:
# Copy yaml file with arguments for second run
cp ../drive/MyDrive/run2.yaml ./data

In [ ]:
# Train YOLOv5s on custom data
!python train.py --img 320 --batch 8 --epochs 60 --data run2.yaml --weights yolov5s.pt --cache

# Setup Model 3 Multiple Pill Images

In [ ]:
# check if files for this run are complete
index = True
for i in range(0, 10):
  for j in range(0, 58):
    for k in range(0, 18):
      if not os.path.isfile("../drive/MyDrive/Data/train3/TestImage" + str(k) + "_" + str(j) + "_" + str(i) + ".jpg"):
        print("../drive/MyDrive/Data/train3/TestImage" + str(k) + "_" + str(j) + "_" + str(i) + ".jpg missing")
        index = False
      if not os.path.isfile("../drive/MyDrive/Data/train3/TestImage" + str(k) + "_" + str(j) + "_" + str(i) + ".txt"):
        print("../drive/MyDrive/Data/train3/TestImage" + str(k) + "_" + str(j) + "_" + str(i) + ".txt missing")
        index = False

for i in range(0, 10000):
   if not os.path.isfile("../drive/MyDrive/Data/train3/TestImage_" + str(i) + ".jpg"):
     print("../drive/MyDrive/Data/train3/TestImage_" + str(i) + ".jpg missing")
     index = False
   if not os.path.isfile("../drive/MyDrive/Data/train3/TestImage_" + str(i) + ".txt"):
     print("../drive/MyDrive/Data/train3/TestImage_" + str(i) + ".txt missing")
     index = False

for i in range(0, 58):
  for j in range(0, 4):
    if not os.path.isfile("../drive/MyDrive/Data/train3/TestImage_E" + str(i) + "_" + str(j) + ".jpg"):
     print("../drive/MyDrive/Data/train3/TestImage_E" + str(i) + "_" + str(j) + ".jpg missing")
     index = False
    if not os.path.isfile("../drive/MyDrive/Data/train3/TestImage_E" + str(i) + "_" + str(j) + ".txt"):
     print("../drive/MyDrive/Data/train3/TestImage_E" + str(i) + "_" + str(j) + ".txt missing")
     index = False

if index:
  print("All " + str(train3_count) + " images found")

All 41344 images found


In [ ]:
# Copy yaml file with arguments for third run
cp ../drive/MyDrive/run3.yaml ./data

In [ ]:
# Train YOLOv5s on custom data
!python train.py --img 320 --batch 8 --epochs 60 --data run3.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=run3.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=60, batch_size=8, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache_images=True, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/ultralytics/yolov5
   6a9846d..a6f9165  update/tf_export -> origin/update/tf_export
github: ⚠️ WARNING: code is out of date by 121 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 🚀 v5.0-304-g2

# Export as ONNX


In [ ]:
# Export trained model as ONNx
!pip install coremltools>=4.1 onnx==1.9.0 scikit-learn==0.19.2  # export requirements
!python export.py --weights runs/train/exp/weights/best.pt --img 320 --batch 1  # export at 640x640 with batch size 1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 0.9.1 requires scikit-learn>=0.20, but you have scikit-learn 0.19.2 which is incompatible.
imbalanced-learn 0.4.3 requires scikit-learn>=0.20, but you have scikit-learn 0.19.2 which is incompatible.
export: weights=runs/train/exp/weights/best.pt, img_size=[320], batch_size=1, device=cpu, include=['torchscript', 'onnx', 'coreml'], half=False, inplace=False, train=False, optimize=False, dynamic=False, simplify=False, opset_version=12
YOLOv5 🚀 v5.0-304-g2c073cd torch 1.9.0+cu102 CPU

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImp

In [ ]:
cd yolov5/

/content/yolov5
